# What this jupyternotebook for?

- ....

# Prepare ENV/Load libraries

## Mount Google Drive

In [1]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
google_drive_path = "/content/gdrive/MyDrive/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# change directory to the project path

project_path = os.path.join(google_drive_path, "Colab Notebooks/SER/")
os.chdir(project_path)
os.listdir("./")

['Notebook Template Train.ipynb',
 'utils',
 'fonts',
 'meta_data',
 'dataset',
 '.git',
 '.env',
 'libs',
 'README.md',
 'models',
 '.gitignore',
 'logs',
 'SER Model idea.drawio',
 'deprecated',
 'Untitled0.ipynb',
 'Preprocess Json to CSV.ipynb',
 'Data distribution.ipynb',
 'Notebook Template.ipynb']

## Install libraries

In [3]:
# !rm -rf tmp && git clone https://github.com/chuan-khuna/my-python-utils.git tmp && cp -R tmp/utils ./ && cp -R tmp/fonts ./ && rm -rf tmp

In [4]:
!python3 --version

Python 3.8.16


Install latest tensorflow version

In [5]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.13.0 requires flatbuffers<3.0,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.
onnx 1.13.0 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.


In [6]:
!pip install matplotlib seaborn -Uq
!pip install pythainlp -q
!pip install keras-tuner -q
!pip install pyYAML -q
!pip install tensorflow-io[tensorflow] tf2onnx onnxruntime -Uq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.11.0 requires tensorflow<2.12,>=2.11.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.9.0 which is incompatible.


In [7]:
%load_ext tensorboard

# Import Libraries

In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import patheffects
import seaborn as sns

import json
import yaml
import re
from tqdm.notebook import trange, tqdm

import datetime

import tf2onnx
import onnx
import onnxruntime as rt

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner
import tensorflow_io as tfio

seed_ = 20200218
tf.random.set_seed(seed_)
np.random.seed(seed_)

from sklearn.metrics import confusion_matrix, classification_report

In [10]:
from utils.vis_utils import *

font_dir = [f"{google_drive_path}/code_assets/fonts/"]
mpl_import_fonts(font_dir)

## Ensure that matplotlib can use subplot mosaic

In [11]:
def check_version(version_str, major, minor):
    print(version_str)
    version = [int(i) for i in version_str.split('.')]
    assert version[0] >= major and version[1] >= minor

check_version(matplotlib.__version__, 3, 6)
check_version(sns.__version__, 0, 12)

del check_version

matplotlib.__version__, sns.__version__, tf.__version__

3.6.2
0.12.2


('3.6.2', '0.12.2', '2.9.0')

## View hardware spec

In [12]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [13]:
tf.config.list_physical_devices('GPU')

for device in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(device, True)

In [14]:
def get_timestamp():
    return datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%d-%H%M")

get_timestamp()

'20221231-1000'

# Load Dataset

In [15]:
df = pd.read_csv("./meta_data/dataset.csv")

# drop emotion label that tagged as None
df = df[df['majority_emo'] != 'None']
df = df[df['majority_emo'] != 'other']
df = df[df['duration'] > 0]

df = df[df['agreement'] >= 0.5]

# shuffle
df = df.sample(frac=1)

In [16]:
assert (df['duration'] > 0).all()
df['majority_emo'].drop_duplicates()

26212           Sad
9305          Happy
22062    Frustrated
1769        Neutral
16102         Angry
Name: majority_emo, dtype: object

In [17]:
df

,path,file,assigned_emo,majority_emo,agreement,duration
26212,./dataset/studio61-70/studio070/con/s070_con_a...,s070_con_actor179_impro15_13.flac,Sad,Sad,1.00,1.738
9305,./dataset/zoom1-10/zoom004/mic/z004_mic_actor0...,z004_mic_actor044_script3_2_3b.flac,Happy,Happy,0.80,5.223
22062,./dataset/studio71-80/studio077/con/s077_con_a...,s077_con_actor194_script2_2_4b.flac,Sad,Frustrated,0.80,6.888
20168,./dataset/studio51-60/studio058/con/s058_con_a...,s058_con_actor156_script3_2_4b.flac,Sad,Sad,1.00,9.238
1769,./dataset/zoom11-20/zoom016/mic/z016_mic_actor...,z016_mic_actor068_script3_2_4a.flac,Sad,Neutral,0.60,4.863
...,...,...,...,...,...,...
5380,./dataset/studio1-10/studio007/con/s007_con_ac...,s007_con_actor014_script1_1_4a.flac,Sad,Neutral,0.75,5.130
18843,./dataset/studio41-50/studio048/con/s048_con_a...,s048_con_actor136_script3_2_2b.flac,Angry,Angry,0.60,4.876
1447,./dataset/zoom11-20/zoom015/mic/z015_mic_actor...,z015_mic_actor066_script3_1_1b.flac,Neutral,Neutral,0.75,6.011
10749,./dataset/zoom1-10/zoom001/mic/z001_mic_actor0...,z001_mic_actor037_script3_1_5b.flac,Frustrated,Angry,0.50,8.350


Copy dataset from Google Drive to VM storage

In [18]:
# !rm -rf /content/dataset/

In [19]:
drive_ds_path = "./dataset/"

target = '/content/'
target_ds_path = '/content/dataset/'

# copy files if not exist
if not(os.path.exists(target_ds_path) and len(os.listdir(target_ds_path)) == 10):
    print("Copy files from Google Drive to VM storage")
    os.system(f'cp -R {drive_ds_path} {target}')

print(os.listdir(target_ds_path))
print(len(os.listdir(target_ds_path)))

['studio11-20', 'studio41-50', 'studio1-10', 'zoom1-10', 'zoom11-20', 'studio51-60', 'studio71-80', 'studio61-70', 'studio31-40', 'studio21-30']
10


In [20]:
df['new_path'] = df['path'].apply(lambda filename: f"/content/{filename[2:]}")

In [21]:
df

,path,file,assigned_emo,majority_emo,agreement,duration,new_path
26212,./dataset/studio61-70/studio070/con/s070_con_a...,s070_con_actor179_impro15_13.flac,Sad,Sad,1.00,1.738,/content/dataset/studio61-70/studio070/con/s07...
9305,./dataset/zoom1-10/zoom004/mic/z004_mic_actor0...,z004_mic_actor044_script3_2_3b.flac,Happy,Happy,0.80,5.223,/content/dataset/zoom1-10/zoom004/mic/z004_mic...
22062,./dataset/studio71-80/studio077/con/s077_con_a...,s077_con_actor194_script2_2_4b.flac,Sad,Frustrated,0.80,6.888,/content/dataset/studio71-80/studio077/con/s07...
20168,./dataset/studio51-60/studio058/con/s058_con_a...,s058_con_actor156_script3_2_4b.flac,Sad,Sad,1.00,9.238,/content/dataset/studio51-60/studio058/con/s05...
1769,./dataset/zoom11-20/zoom016/mic/z016_mic_actor...,z016_mic_actor068_script3_2_4a.flac,Sad,Neutral,0.60,4.863,/content/dataset/zoom11-20/zoom016/mic/z016_mi...
...,...,...,...,...,...,...,...
5380,./dataset/studio1-10/studio007/con/s007_con_ac...,s007_con_actor014_script1_1_4a.flac,Sad,Neutral,0.75,5.130,/content/dataset/studio1-10/studio007/con/s007...
18843,./dataset/studio41-50/studio048/con/s048_con_a...,s048_con_actor136_script3_2_2b.flac,Angry,Angry,0.60,4.876,/content/dataset/studio41-50/studio048/con/s04...
1447,./dataset/zoom11-20/zoom015/mic/z015_mic_actor...,z015_mic_actor066_script3_1_1b.flac,Neutral,Neutral,0.75,6.011,/content/dataset/zoom11-20/zoom015/mic/z015_mi...
10749,./dataset/zoom1-10/zoom001/mic/z001_mic_actor0...,z001_mic_actor037_script3_1_5b.flac,Frustrated,Angry,0.50,8.350,/content/dataset/zoom1-10/zoom001/mic/z001_mic...
